Cryptodome Digital Signature

In [3]:
import libcst as cst

from libcst.display import dump

code1 = """
from Crypto.PublicKey import RSA
    
from Crypto.Signature import pkcs1_15
from Crypto.Hash import SHA256

# Generate RSA key pair
key = RSA.generate(1024)
private_key = key
public_key = key.publickey()

# Message to sign
message = b"Hello, this is a message to sign!"

# Create a SHA256 hash of the message
hash_obj = SHA256.new(message)

# Sign the message with the private key
signature = pkcs1_15.new(private_key).sign(hash_obj)

# Verify the signature with the public key
try:
    pkcs1_15.new(public_key).verify(hash_obj, signature)
    print("Signature is valid.")
except (ValueError, TypeError):
    print("Signature is invalid.")

# Save keys and signature (optional)
with open("rsa_private.pem", "wb") as f:
    f.write(private_key.export_key())
with open("rsa_public.pem", "wb") as f:
    f.write(public_key.export_key())
with open("signature.bin", "wb") as f:
    f.write(signature)
"""

tree = cst.parse_module(code1)
print(dump(tree))

Module(
  body=[
    SimpleStatementLine(
      body=[
        ImportFrom(
          module=Attribute(
            value=Name(
              value='Crypto',
            ),
            attr=Name(
              value='PublicKey',
            ),
          ),
          names=[
            ImportAlias(
              name=Name(
                value='RSA',
              ),
            ),
          ],
        ),
      ],
    ),
    SimpleStatementLine(
      body=[
        ImportFrom(
          module=Attribute(
            value=Name(
              value='Crypto',
            ),
            attr=Name(
              value='Signature',
            ),
          ),
          names=[
            ImportAlias(
              name=Name(
                value='pkcs1_15',
              ),
            ),
          ],
        ),
      ],
    ),
    SimpleStatementLine(
      body=[
        ImportFrom(
          module=Attribute(
            value=Name(
              value='Crypto',
            ),
       

In [4]:
from arta import RulesEngine
inputs = {
    "tree":tree
}


libname = "cryptography"

eng = RulesEngine(config_path="./arta_files")

extracted_result = eng.apply_rules(inputs, rule_set="mapping_creator_rule_set")
mappings = extracted_result['get_mapping_ids']

print(f"Mappings:{mappings}")

if "public_key" not in mappings.keys():
    mappings["public_key"] = "public_" + mappings["private_key"]

mod_inputs = {
    "tree":tree,
    "mappings":mappings
}

result = eng.apply_rules(mod_inputs, rule_set="keygen_rule_set")
modified_keygen_tree = result['check_generate']
mod_inputs2 = {
    "tree":modified_keygen_tree,
    "mappings":mappings
}

primitive_result = eng.apply_rules(mod_inputs, rule_set="ds_primitive_rule_set")
modified_primitive_tree = primitive_result['check_primitive']


mod_inputs3 = {
    "tree":modified_primitive_tree,
    "mappings":mappings
}

result = eng.apply_rules(mod_inputs3, rule_set="ds_sign_rule_set")
modified_sign_tree = result['check_sign']


mod_inputs4 = {
    "tree":modified_sign_tree,
    "library": libname
}

verify_result = eng.apply_rules(mod_inputs4, rule_set="ds_verify_rule_set")
modified_verify_tree = verify_result['check_verify']



mod_inputs5 = {
    "tree":modified_verify_tree,
    "mappings":mappings
}

remove_result = eng.apply_rules(mod_inputs5, rule_set="remove_rule_set")
remove_pubkey_tree = remove_result['check_remove_pubkey']

mod_inputs6 = {
    "tree":remove_pubkey_tree,
    "mappings":mappings
}

add_result = eng.apply_rules(mod_inputs6, rule_set="add_rule_set")
add_hash_nodes_tree = add_result['insert_hash_nodes']


mod_inputs7 = {
    "tree":add_hash_nodes_tree,
    "mappings":mappings,
    "library":libname
}

import_result = eng.apply_rules(mod_inputs7, rule_set="update_imports_rule_set")
update_imports_tree = import_result['check_imports']



print("----OUTPUT---")
#print(modified_keygen_tree.code)
print(update_imports_tree.code)

ConditionExecutionError: Error while executing condition 'CHECK_IMPORT_CRYPTOGRAPHY': "Could not find path 'library' in the input data: 'library'"

NACL digital signature

In [3]:
import libcst as cst

from libcst.display import dump

code1 = """
from nacl.signing import SigningKey

# Generate a new random signing key
signing_key = SigningKey.generate()

# Sign a message with the signing key
signed = signing_key.sign(b"Attack at Dawn")

# Obtain the verify key for a given signing key
verify_key = signing_key.verify_key

# Serialize the verify key to send it to a third party
verify_key_bytes = verify_key.encode()
"""


tree = cst.parse_module(code1)
#print(dump(tree))
print(tree.code)




from nacl.signing import SigningKey

# Generate a new random signing key
signing_key = SigningKey.generate()

# Sign a message with the signing key
signed = signing_key.sign(b"Attack at Dawn")

# Obtain the verify key for a given signing key
verify_key = signing_key.verify_key

# Serialize the verify key to send it to a third party
verify_key_bytes = verify_key.encode()



In [4]:
from arta import RulesEngine


inputs = {
    "tree":tree
}


libname = "nacl"

eng = RulesEngine(config_path="./arta_files")

extracted_result = eng.apply_rules(inputs, rule_set="mapping_creator_rule_set")
mappings = extracted_result['get_mapping_ids']



if "public_key" not in mappings.keys():
    mappings["public_key"] = "public_" + mappings["private_key"]

print(f"Mappings:{mappings}")


mod_inputs = {
    "tree":tree,
    "mappings":mappings,
    "library": libname
}

result = eng.apply_rules(mod_inputs, rule_set="keygen_rule_set")
modified_keygen_tree = result['check_generate']

mod_inputs2 = {
    "tree":modified_keygen_tree,
    "mappings":mappings,
    "library":libname
}




result = eng.apply_rules(mod_inputs2, rule_set="ds_sign_rule_set")
modified_sign_tree = result['check_sign']


"""
mod_inputs3 = {
    "tree":modified_sign_tree,
    "mappings": mappings,
    "library":libname
}

verify_result = eng.apply_rules(mod_inputs3, rule_set="ds_verify_rule_set")
modified_verify_tree = verify_result['check_verify']
"""


print("**OUTPUT**\n")
print(modified_sign_tree.code)



#need to add import statements and remove classical and add pqc ones also make sure import statements are unique




['nacl.signing.SigningKey.generate'] names
['signing_key.sign'] names
['verify_key.encode'] names
Mappings:{'private_key': 'signing_key', 'message': 'b"Attack at Dawn"', 'signing_key': 'signing_key', 'public_key': 'public_signing_key'}
False
in action keygne nacl

qnames  []
qnames  ['signing_key.sign']
qnames  ['verify_key.encode']
**OUTPUT**


from quantcrypt.dss import MLDSA_65


# Generate a new random signing key
public_signing_key, signing_key = MLDSA_65().keygen()

# Sign a message with the signing key
signed = MLDSA_65().sign(signing_key, b"Attack at Dawn")

# Obtain the verify key for a given signing key
verify_key = public_signing_key

# Serialize the verify key to send it to a third party
verify_key_bytes = verify_key.encode()



In [ ]:
print(modified_sign_tree.code)


from quantcrypt.dss import MLDSA_65
# Generate a new random signing key
public_signing_key, signing_key = MLDSA_65().keygen()

# Sign a message with the signing key
signed = MLDSA_65().sign(signing_key, b"Attack at Dawn")

# Obtain the verify key for a given signing key
verify_key = signing_key.verify_key

# Serialize the verify key to send it to a third party
verify_key_bytes = verify_key.encode()



In [5]:
#KEM


import libcst as cst

from libcst.display import dump

code = """
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric.x25519 import X25519PrivateKey
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
# Generate a private key for use in the exchange.
private_key = X25519PrivateKey.generate()
# In a real handshake the peer_public_key will be received from the
# other party. For this example we'll generate another private key and
# get a public key from that. Note that in a DH handshake both peers
# must agree on a common set of parameters.
peer_public_key = X25519PrivateKey.generate().public_key()
shared_key = private_key.exchange(peer_public_key)
# Perform key derivation.
derived_key = HKDF(
    algorithm=hashes.SHA256(),
    length=32,
    salt=None,
    info=b'handshake data',
).derive(shared_key)


"""



tree = cst.parse_module(code)
#print(dump(tree))
print(tree.code)


from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric.x25519 import X25519PrivateKey
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
# Generate a private key for use in the exchange.
private_key = X25519PrivateKey.generate()
# In a real handshake the peer_public_key will be received from the
# other party. For this example we'll generate another private key and
# get a public key from that. Note that in a DH handshake both peers
# must agree on a common set of parameters.
peer_public_key = X25519PrivateKey.generate().public_key()
shared_key = private_key.exchange(peer_public_key)
# Perform key derivation.
derived_key = HKDF(
    algorithm=hashes.SHA256(),
    length=32,
    salt=None,
    info=b'handshake data',
).derive(shared_key)





In [7]:
from arta import RulesEngine


libname = "cryptography"

inputs = {
    "tree":tree,
    "library":libname
}


eng = RulesEngine(config_path="./arta_files")

extracted_result = eng.apply_rules(inputs, rule_set="mapping_creator_rule_set")
mappings = extracted_result['get_mapping_ids']




print(f"Mappings:{mappings}")

libname = "cryptography"
mod_inputs = {
    "tree":tree,
    "mappings":mappings,
    "library":libname
    
}

result = eng.apply_rules(mod_inputs, rule_set="kem_keygen_rule_set")
modified_keygen_tree = result['check_generate']


print(modified_keygen_tree.code)


True
Mappings:{'algorithm': 'x25519', 'kem_private_generate': 'private_key', 'peer_public_key': ['peer_public_key', 'peer_public_key'], 'hash_algo': 'SHA256', 'digest_id': 'digest'}
True
['cryptography.hazmat.primitives.asymmetric.x25519.X25519PrivateKey.generate']
['cryptography.hazmat.primitives.asymmetric.x25519.X25519PrivateKey.generate.public_key']
['private_key.exchange']
['cryptography.hazmat.primitives.kdf.hkdf.HKDF.derive']

from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from quantcrypt.kem import MLKEM_512

# Generate a private key for use in the exchange.
public_key_kem, private_key = MLKEM_512().keygen()
# In a real handshake the peer_public_key will be received from the
# other party. For this example we'll generate another private key and
# get a public key from that. Note that in a DH handshake both peers
# must agree on a common set of parameters.
peer_public_key, peer_public_key_priv = MLKEM_512().keygen()
cip

RSA digital signatuere

In [8]:
import libcst as cst

from libcst.display import dump



code = """from cryptography.hazmat.primitives.asymmetric import rsa
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
)"""
tree = cst.parse_module(code)
#print(dump(tree))
print(tree.code)

from cryptography.hazmat.primitives.asymmetric import rsa
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
)


In [10]:
from arta import RulesEngine


libname = "cryptography"
inputs = {
    "tree":tree,
    "library":libname
}



eng = RulesEngine(config_path="./arta_files")

extracted_result = eng.apply_rules(inputs, rule_set="mapping_creator_rule_set")
mappings = extracted_result['get_mapping_ids']



if "public_key" not in mappings.keys():
    mappings["public_key"] = "public_" + mappings["private_key"]

print(f"Mappings:{mappings}")

libname = "cryptography"
mod_inputs = {
    "tree":tree,
    "mappings":mappings,
    "library":libname
    
}

result = eng.apply_rules(mod_inputs, rule_set="keygen_rule_set")
modified_keygen_tree = result['check_generate']


print(modified_keygen_tree.code)


True
Mappings:{'algorithm': 'rsa', 'private_key': 'private_key', 'public_key': 'public_private_key'}
True
keygen cryptography

public_private_key, private_key = MLDSA_87().keygen()


In [11]:
import libcst as cst

from libcst.display import dump


code="""from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import dsa
private_key = dsa.generate_private_key(
    key_size=1024,
)
data = b"this is some data I'd like to sign"
signature = private_key.sign(
    data,
    hashes.SHA256()
)"""



tree = cst.parse_module(code)
#print(dump(tree))
print(tree.code)

from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import dsa
private_key = dsa.generate_private_key(
    key_size=1024,
)
data = b"this is some data I'd like to sign"
signature = private_key.sign(
    data,
    hashes.SHA256()
)


In [12]:
from arta import RulesEngine


libname = "cryptography"
inputs = {
    "tree":tree,
    "library":libname
}



eng = RulesEngine(config_path="./arta_files")

extracted_result = eng.apply_rules(inputs, rule_set="mapping_creator_rule_set")
mappings = extracted_result['get_mapping_ids']


if "public_key" not in mappings.keys():
    mappings["public_key"] = "public_" + mappings["private_key"]

print(f"Mappings:{mappings}")

libname = "cryptography"
mod_inputs = {
    "tree":tree,
    "mappings":mappings,
    "library":libname
    
}

result = eng.apply_rules(mod_inputs, rule_set="keygen_rule_set")
modified_keygen_tree = result['check_generate']

print(modified_keygen_tree.code)


True
Mappings:{'algorithm': 'dsa', 'private_key': 'private_key', 'hash_algo': 'SHA256', 'digest_id': 'digest', 'public_key': 'public_private_key'}
True
keygen cryptography

from cryptography.hazmat.primitives import hashes

public_private_key, private_key = MLDSA_65().keygen()
data = b"this is some data I'd like to sign"
signature = private_key.sign(
    data,
    hashes.SHA256()
)


KEM 

Crytography

In [13]:
import libcst as cst

from libcst.display import dump


code = """ 
import sys
import io
import os
import shutil
import hashlib
from cryptography.hazmat.primitives.asymmetric.x25519 import X25519PrivateKey
from cryptography.hazmat.primitives.asymmetric.x25519 import X25519PublicKey
import sosemanuk

RANSOM_EXT = '.babyk'
MAX_ENC_SIZE = (0x20000000 // 0xA00000 + 1) * 0xA00000
X25519_KEY_SIZE = 32
SOSEMANUK_KEY_SIZE = 32
METADATA_SIZE = X25519_KEY_SIZE
ENC_BLOCK_SIZE = 0x100000


def decrypt_file(filename: str, priv_key_data: bytes) -> bool:

    with io.open(filename, 'rb+') as f:

        # Read metadata
        try:
            f.seek(-METADATA_SIZE, 2)
        except OSError:
            return False

        pub_key_data = f.read(METADATA_SIZE)
        # Derive x25519 shared secret
        priv_key = X25519PrivateKey.from_private_bytes(priv_key_data)
        pub_key = X25519PublicKey.from_public_bytes(pub_key_data)
        shared_secret = priv_key.exchange(pub_key)
        # Derive Sosemanuk encryption key
        key = hashlib.sha256(shared_secret).digest()
        # Remove metadata
        f.seek(-METADATA_SIZE, 2)
        f.truncate()
        cipher = sosemanuk.Sosemanuk(key)
        f.seek(0)
        total_read = 0

        while total_read < MAX_ENC_SIZE:

            enc_data = f.read(ENC_BLOCK_SIZE)
            if enc_data == b'':
                break

            data = cipher.decrypt(enc_data)

            bytes_read = len(enc_data)
            total_read += bytes_read

            f.seek(-bytes_read, 1)
            f.write(data)
        
    return True

if len(sys.argv) != 2:
    print('Usage:', os.path.basename(sys.argv[0]), 'filename')
    sys.exit(0)
filename = sys.argv[1]

with io.open('./privkey.bin', 'rb') as f:
    priv_key_data = f.read()

new_filename = filename
if new_filename.endswith(RANSOM_EXT):
    new_filename = new_filename[:-len(RANSOM_EXT)]
else:
    new_filename += '.dec'
shutil.copy(filename, new_filename)
if not decrypt_file(new_filename, priv_key_data):
    os.remove(new_filename)
    print('Error: Failed to decrypt file')
    sys.exit(1)
"""


tree = cst.parse_module(code)
#print(dump(tree))
print(tree.code)

 
import sys
import io
import os
import shutil
import hashlib
from cryptography.hazmat.primitives.asymmetric.x25519 import X25519PrivateKey
from cryptography.hazmat.primitives.asymmetric.x25519 import X25519PublicKey
import sosemanuk

RANSOM_EXT = '.babyk'
MAX_ENC_SIZE = (0x20000000 // 0xA00000 + 1) * 0xA00000
X25519_KEY_SIZE = 32
SOSEMANUK_KEY_SIZE = 32
METADATA_SIZE = X25519_KEY_SIZE
ENC_BLOCK_SIZE = 0x100000


def decrypt_file(filename: str, priv_key_data: bytes) -> bool:

    with io.open(filename, 'rb+') as f:

        # Read metadata
        try:
            f.seek(-METADATA_SIZE, 2)
        except OSError:
            return False

        pub_key_data = f.read(METADATA_SIZE)
        # Derive x25519 shared secret
        priv_key = X25519PrivateKey.from_private_bytes(priv_key_data)
        pub_key = X25519PublicKey.from_public_bytes(pub_key_data)
        shared_secret = priv_key.exchange(pub_key)
        # Derive Sosemanuk encryption key
        key = hashlib.sha256(shared_secre

In [14]:
from arta import RulesEngine


libname = "cryptography"
inputs = {
    "tree":tree,
    "library":libname
}



eng = RulesEngine(config_path="./arta_files")

extracted_result = eng.apply_rules(inputs, rule_set="mapping_creator_rule_set")
mappings = extracted_result['get_mapping_ids']


print("mappings ", mappings)

print(f"Mappings:{mappings}")

libname = "cryptography"
mod_inputs = {
    "tree":tree,
    "mappings":mappings,
    "library":libname
    
}

result = eng.apply_rules(mod_inputs, rule_set="kem_keygen_rule_set")
modified_keygen_tree = result['check_generate']


print(modified_keygen_tree.code)



True
mappings  {'algorithm': 'x448', 'private_key': 'priv_key', 'priv_key_bytes': 'priv_key_data', 'public_key': 'pub_key', 'pub_key_bytes': 'pub_key_data', 'peer_public_key': 'pub_key'}
Mappings:{'algorithm': 'x448', 'private_key': 'priv_key', 'priv_key_bytes': 'priv_key_data', 'public_key': 'pub_key', 'pub_key_bytes': 'pub_key_data', 'peer_public_key': 'pub_key'}
True
['decrypt_file.<locals>.f.read']
['cryptography.hazmat.primitives.asymmetric.x25519.X25519PrivateKey.from_private_bytes']
['cryptography.hazmat.primitives.asymmetric.x25519.X25519PublicKey.from_public_bytes']
['decrypt_file.<locals>.priv_key.exchange']
['hashlib.sha256.digest']
['sosemanuk.Sosemanuk']
['decrypt_file.<locals>.f.read']
['decrypt_file.<locals>.cipher.decrypt']
['builtins.len']
['f.read']
 
import sys
import io
import os
import shutil
import hashlib
import sosemanuk
from quantcrypt.kem import MLKEM_512


RANSOM_EXT = '.babyk'
MAX_ENC_SIZE = (0x20000000 // 0xA00000 + 1) * 0xA00000
X25519_KEY_SIZE = 32
SOSEMA

In [15]:
#KEM


import libcst as cst

from libcst.display import dump

code = """
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric.x25519 import X25519PrivateKey
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
# Generate a private key for use in the exchange.
private_key = X25519PrivateKey.generate()
# In a real handshake the peer_public_key will be received from the
# other party. For this example we'll generate another private key and
# get a public key from that. Note that in a DH handshake both peers
# must agree on a common set of parameters.
peer_public_key = X25519PrivateKey.generate().public_key()
shared_key = private_key.exchange(peer_public_key)
# Perform key derivation.
derived_key = HKDF(
    algorithm=hashes.SHA256(),
    length=32,
    salt=None,
    info=b'handshake data',
).derive(shared_key)


"""



tree = cst.parse_module(code)
#print(dump(tree))
print(tree.code)


from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric.x25519 import X25519PrivateKey
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
# Generate a private key for use in the exchange.
private_key = X25519PrivateKey.generate()
# In a real handshake the peer_public_key will be received from the
# other party. For this example we'll generate another private key and
# get a public key from that. Note that in a DH handshake both peers
# must agree on a common set of parameters.
peer_public_key = X25519PrivateKey.generate().public_key()
shared_key = private_key.exchange(peer_public_key)
# Perform key derivation.
derived_key = HKDF(
    algorithm=hashes.SHA256(),
    length=32,
    salt=None,
    info=b'handshake data',
).derive(shared_key)





In [16]:
from arta import RulesEngine


inputs = {
    "tree":tree,
    "library":libname
}


libname = "cryptography"

eng = RulesEngine(config_path="./arta_files")

extracted_result = eng.apply_rules(inputs, rule_set="mapping_creator_rule_set")
mappings = extracted_result['get_mapping_ids']




print(f"Mappings:{mappings}")

libname = "cryptography"
mod_inputs = {
    "tree":tree,
    "mappings":mappings,
    "library":libname
    
}

result = eng.apply_rules(mod_inputs, rule_set="kem_keygen_rule_set")
modified_keygen_tree = result['check_generate']


print(modified_keygen_tree.code)


True
Mappings:{'algorithm': 'x25519', 'kem_private_generate': 'private_key', 'peer_public_key': ['peer_public_key', 'peer_public_key'], 'hash_algo': 'SHA256', 'digest_id': 'digest'}
True
['cryptography.hazmat.primitives.asymmetric.x25519.X25519PrivateKey.generate']
['cryptography.hazmat.primitives.asymmetric.x25519.X25519PrivateKey.generate.public_key']
['private_key.exchange']
['cryptography.hazmat.primitives.kdf.hkdf.HKDF.derive']

from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from quantcrypt.kem import MLKEM_512

# Generate a private key for use in the exchange.
public_key_kem, private_key = MLKEM_512().keygen()
# In a real handshake the peer_public_key will be received from the
# other party. For this example we'll generate another private key and
# get a public key from that. Note that in a DH handshake both peers
# must agree on a common set of parameters.
peer_public_key, peer_public_key_priv = MLKEM_512().keygen()
cip

In [17]:
import libcst as cst

from libcst.display import dump
code = """
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric.x448 import X448PrivateKey
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
# Generate a private key for use in the exchange.
private_key = X448PrivateKey.generate()
# In a real handshake the peer_public_key will be received from the
# other party. For this example we'll generate another private key and
# get a public key from that. Note that in a DH handshake both peers
# must agree on a common set of parameters.
peer_public_key = X448PrivateKey.generate().public_key()
shared_key = private_key.exchange(peer_public_key)
# Perform key derivation.
derived_key = HKDF(
    algorithm=hashes.SHA256(),
    length=32,
    salt=None,
    info=b'handshake data',
).derive(shared_key)
# For the next handshake we MUST generate another private key.
private_key_2 = X448PrivateKey.generate()
peer_public_key_2 = X448PrivateKey.generate().public_key()
shared_key_2 = private_key_2.exchange(peer_public_key_2)
derived_key_2 = HKDF(
    algorithm=hashes.SHA256(),
    length=32,
    salt=None,
    info=b'handshake data',
).derive(shared_key_2)"""

tree = cst.parse_module(code)
#print(dump(tree))
print(tree.code)


from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric.x448 import X448PrivateKey
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
# Generate a private key for use in the exchange.
private_key = X448PrivateKey.generate()
# In a real handshake the peer_public_key will be received from the
# other party. For this example we'll generate another private key and
# get a public key from that. Note that in a DH handshake both peers
# must agree on a common set of parameters.
peer_public_key = X448PrivateKey.generate().public_key()
shared_key = private_key.exchange(peer_public_key)
# Perform key derivation.
derived_key = HKDF(
    algorithm=hashes.SHA256(),
    length=32,
    salt=None,
    info=b'handshake data',
).derive(shared_key)
# For the next handshake we MUST generate another private key.
private_key_2 = X448PrivateKey.generate()
peer_public_key_2 = X448PrivateKey.generate().public_key()
shared_key_2 = private_key_2.exchange(peer_publi

In [7]:
from arta import RulesEngine

libname = "cryptography"

inputs = {
    "tree":tree,
    "library":libname
}



eng = RulesEngine(config_path="./arta_files")

extracted_result = eng.apply_rules(inputs, rule_set="mapping_creator_rule_set")
mappings = extracted_result['get_mapping_ids']




print(f"Mappings:{mappings}")

libname = "cryptography"
mod_inputs = {
    "tree":tree,
    "mappings":mappings,
    "library":libname
    
}

result = eng.apply_rules(mod_inputs, rule_set="kem_keygen_rule_set")
modified_keygen_tree = result['check_generate']


print(modified_keygen_tree.code)


True
Mappings:{'algorithm': 'X448', 'peer_public_key': ['peer_public_key_2', 'peer_public_key_2'], 'hash_algo': 'SHA256', 'digest_id': 'digest'}
True
['cryptography.hazmat.primitives.asymmetric.x448.X448PrivateKey.generate']
['cryptography.hazmat.primitives.asymmetric.x448.X448PrivateKey.generate.public_key']
['private_key.exchange']
['cryptography.hazmat.primitives.kdf.hkdf.HKDF.derive']
['cryptography.hazmat.primitives.asymmetric.x448.X448PrivateKey.generate']
['cryptography.hazmat.primitives.asymmetric.x448.X448PrivateKey.generate.public_key']
['private_key_2.exchange']
['cryptography.hazmat.primitives.kdf.hkdf.HKDF.derive']

from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from quantcrypt.kem import MLKEM_512# Generate a private key for use in the exchange.
private_key = X448PrivateKey.generate()
# In a real handshake the peer_public_key will be received from the
# other party. For this example we'll generate another private

In [8]:
import libcst as cst

from libcst.display import dump
code = """
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
# Generate some parameters. These can be reused.
parameters = dh.generate_parameters(generator=2, key_size=2048)
# Generate a private key for use in the exchange.
server_private_key = parameters.generate_private_key()
# In a real handshake the peer is a remote client. For this
# example we'll generate another local private key though. Note that in
# a DH handshake both peers must agree on a common set of parameters.
peer_private_key = parameters.generate_private_key()
shared_key = server_private_key.exchange(peer_private_key.public_key())
# Perform key derivation.
derived_key = HKDF(
    algorithm=hashes.SHA256(),
    length=32,
    salt=None,
    info=b'handshake data',
).derive(shared_key)
# And now we can demonstrate that the handshake performed in the
# opposite direction gives the same final value
same_shared_key = peer_private_key.exchange(
    server_private_key.public_key()
)
same_derived_key = HKDF(
    algorithm=hashes.SHA256(),
    length=32,
    salt=None,
    info=b'handshake data',
).derive(same_shared_key)
derived_key == same_derived_key
"""

tree = cst.parse_module(code)
#print(dump(tree))
print(tree.code)


from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
# Generate some parameters. These can be reused.
parameters = dh.generate_parameters(generator=2, key_size=2048)
# Generate a private key for use in the exchange.
server_private_key = parameters.generate_private_key()
# In a real handshake the peer is a remote client. For this
# example we'll generate another local private key though. Note that in
# a DH handshake both peers must agree on a common set of parameters.
peer_private_key = parameters.generate_private_key()
shared_key = server_private_key.exchange(peer_private_key.public_key())
# Perform key derivation.
derived_key = HKDF(
    algorithm=hashes.SHA256(),
    length=32,
    salt=None,
    info=b'handshake data',
).derive(shared_key)
# And now we can demonstrate that the handshake performed in the
# opposite direction gives the same final value
same_shared_key = pee

In [9]:
from arta import RulesEngine


libname = "cryptography"
mappings = {}
inputs = {
    "tree":tree,
    "library":libname,
    "mappings": mappings
}



eng = RulesEngine(config_path="./arta_files")

extracted_result = eng.apply_rules(inputs, rule_set="mapping_creator_rule_set")
mappings = extracted_result['get_mapping_ids']




print(f"Mappings:{mappings}")

libname = "cryptography"
mod_inputs = {
    "tree":tree,
    "mappings":mappings,
    "library":libname
    
}

result = eng.apply_rules(mod_inputs, rule_set="kem_keygen_rule_set")
modified_keygen_tree = result['check_generate']

#need to update for exchange func with calls param key.publickey()

print(modified_keygen_tree.code)


True
arg 1,,  peer_private_key
arg 1,,  server_private_key
Mappings:{'algorithm': 'dh', 'kem_param': 'parameters', 'kem_key_size': 2048, 'kem_private_generate': 'peer_private_key', 'peer_public_key': ['peer_private_key', 'server_private_key'], 'hash_algo': 'SHA256', 'digest_id': 'digest'}
True
['cryptography.hazmat.primitives.asymmetric.dh.generate_parameters']
['parameters.generate_private_key']
['parameters.generate_private_key']
['server_private_key.exchange']
['cryptography.hazmat.primitives.kdf.hkdf.HKDF.derive']
['peer_private_key.exchange']
['cryptography.hazmat.primitives.kdf.hkdf.HKDF.derive']

from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from quantcrypt.kem import MLKEM_1024# Generate a private key for use in the exchange.
public_key_kem, peer_private_key = MLKEM_1024().keygen()
# In a real handshake the peer is a remote client. For this
# example we'll generate another local private key though. Note that in
# a DH

In [ ]:
#NACL key encryption
#also still need work on cryptography ec key exchange
import libcst as cst

from libcst.display import dump

code = """
import nacl.utils
from nacl.public import PrivateKey, Box

# generate the private key which must be kept secret
skbob = PrivateKey.generate()

# the public key can be given to anyone wishing to send
# Bob an encrypted message
pkbob = skbob.public_key

# Alice does the same and then
#     sends her public key to Bob and Bob his public key to Alice
skalice = PrivateKey.generate()
pkalice = skalice.public_key

# Bob wishes to send Alice an encrypted message
# So Bob must make a Box with his private key and Alice's public key
bob_box = Box(skbob, pkalice)

# This is our message to send, it must be a bytestring as Box will
#   treat is as just a binary blob of data.
message = b"Kill all humans"

# This is a nonce, it *MUST* only be used once, but it is not considered
#   secret and can be transmitted or stored alongside the ciphertext. A
#   good source of nonce is just 24 random bytes.
nonce = nacl.utils.random(Box.NONCE_SIZE)

# Encrypt our message, it will be exactly 40 bytes longer than the original
#   message as it stores authentication information and nonce alongside it.
encrypted = bob_box.encrypt(message, nonce)

# Alice creates a second box with her private key to decrypt the message
alice_box = Box(skalice, pkbob)

# Decrypt our message, an exception will be raised if the encryption was
#   tampered with or there was otherwise an error.
plaintext = alice_box.decrypt(encrypted)"""


tree = cst.parse_module(code)
#print(dump(tree))
print(tree.code)


import nacl.utils
from nacl.public import PrivateKey, Box

# generate the private key which must be kept secret
skbob = PrivateKey.generate()

# the public key can be given to anyone wishing to send
# Bob an encrypted message
pkbob = skbob.public_key

# Alice does the same and then
#     sends her public key to Bob and Bob his public key to Alice
skalice = PrivateKey.generate()
pkalice = skalice.public_key

# Bob wishes to send Alice an encrypted message
# So Bob must make a Box with his private key and Alice's public key
bob_box = Box(skbob, pkalice)

# This is our message to send, it must be a bytestring as Box will
#   treat is as just a binary blob of data.
message = b"Kill all humans"

# This is a nonce, it *MUST* only be used once, but it is not considered
#   secret and can be transmitted or stored alongside the ciphertext. A
#   good source of nonce is just 24 random bytes.
nonce = nacl.utils.random(Box.NONCE_SIZE)

# Encrypt our message, it will be exactly 40 bytes longer tha

Digital Signature

In [1]:

import libcst as cst

from libcst.display import dump

code = """
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import ec
private_key = ec.generate_private_key(
    ec.SECP384R1()
)
data = b"this is some data I'd like to sign"
signature = private_key.sign(
    data,
    ec.ECDSA(hashes.SHA256())
)
"""


tree = cst.parse_module(code)
#print(dump(tree))
print(tree.code)


from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import ec
private_key = ec.generate_private_key(
    ec.SECP384R1()
)
data = b"this is some data I'd like to sign"
signature = private_key.sign(
    data,
    ec.ECDSA(hashes.SHA256())
)



In [2]:
from arta import RulesEngine

libname = "cryptography"

inputs = {
    "tree":tree,
    "library":libname
}



eng = RulesEngine(config_path="./arta_files")

extracted_result = eng.apply_rules(inputs, rule_set="mapping_creator_rule_set")
mappings = extracted_result['get_mapping_ids']



print(f"Mappings:{mappings}")


mod_inputs = {
    "tree":tree,
    "mappings":mappings,
    "library": libname
}

result = eng.apply_rules(mod_inputs, rule_set="keygen_rule_set")
modified_keygen_tree = result['check_generate']

mod_inputs2 = {
    "tree":modified_keygen_tree,
    "mappings":mappings,
    "library":libname
}




result = eng.apply_rules(mod_inputs2, rule_set="ds_sign_rule_set")
modified_sign_tree = result['check_sign']


"""
mod_inputs3 = {
    "tree":modified_sign_tree,
    "mappings": mappings,
    "library":libname
}

verify_result = eng.apply_rules(mod_inputs3, rule_set="ds_verify_rule_set")
modified_verify_tree = verify_result['check_verify']
"""


print("**OUTPUT**\n")
print(modified_sign_tree.code)

True
SECP384R1
Mappings:{'algorithm': 'ec', 'private_key': 'private_key', 'public_key': 'private_key_pub', 'pqc_alg': 'MLDSA_65', 'hash_algo': 'SHA256', 'digest_id': 'digest'}
True
keygen cryptography

['cryptography.hazmat.primitives.asymmetric.ec.generate_private_key'] 
qnames
[] 
qnames
['private_key.sign'] 
qnames
True
**OUTPUT**


from cryptography.hazmat.primitives import hashes
from quantcrypt.dss import MLDSA_65

private_key_pub, private_key = MLDSA_65().keygen()
data = b"this is some data I'd like to sign"
signature = MLDSA_65.sign(private_key, data)



In [2]:

import libcst as cst

from libcst.display import dump

source_code = """
from Crypto.PublicKey import ECC
from Crypto.Hash import SHAKE128
from Crypto.Protocol.DH import key_agreement

# This KDF has been agreed in advance
def kdf(x):
        return SHAKE128.new(x).read(32)

# In a real scenario, this key already exists
U = ECC.generate(curve='p256')

# In a real scenario, this key is received from the peer
# and it is verified as authentic
V = ECC.generate(curve='p256').public_key()

session_key = key_agreement(static_priv=U, static_pub=V, kdf=kdf)
"""

target_code = """ 
from quantcrypt.dss import MLDSA_65
from Crypto.Hash import SHAKE128
def kdf(x):
        return SHAKE128.new(x).read(32)

pub_U, U = MLDSA_65.keygen()
V, priv_V = MLDSA_65.keygen()
cipher_text, shared_key = MLDSA_65.encaps(pub_U)
session_key = kdf(shared_Key)
 """

tree = cst.parse_module(source_code)
#print(dump(tree))
print(tree.code)


from Crypto.PublicKey import ECC
from Crypto.Hash import SHAKE128
from Crypto.Protocol.DH import key_agreement

# This KDF has been agreed in advance
def kdf(x):
        return SHAKE128.new(x).read(32)

# In a real scenario, this key already exists
U = ECC.generate(curve='p256')

# In a real scenario, this key is received from the peer
# and it is verified as authentic
V = ECC.generate(curve='p256').public_key()

session_key = key_agreement(static_priv=U, static_pub=V, kdf=kdf)



In [3]:
from arta import RulesEngine

libname = "cryptodome"

inputs = {
    "tree":tree,
    "library":libname
}



eng = RulesEngine(config_path="./arta_files")

extracted_result = eng.apply_rules(inputs, rule_set="mapping_creator_rule_set")
mappings = extracted_result['get_mapping_ids']

print(mappings)



mod_inputs = {
    "tree":tree,
    "mappings":mappings,
    "library": libname
}

result = eng.apply_rules(mod_inputs, rule_set="kem_keygen_rule_set")
modified_keygen_tree = result['check_generate']

print(modified_keygen_tree.code)



False
Crypto.PublicKey
Crypto.Hash
Crypto.Protocol.DH
has cyrptodome True
names  ['Crypto.PublicKey.ECC.generate']
names  ['Crypto.PublicKey.ECC.generate.public_key']
names  ['Crypto.Protocol.DH.key_agreement']
{'algorithm': 'ECC', 'private_key': 'U'}
False
Crypto.PublicKey
Crypto.Hash
Crypto.Protocol.DH
has cyrptodome True

from Crypto.PublicKey import ECC
from Crypto.Hash import SHAKE128
from Crypto.Protocol.DH import key_agreement


# This KDF has been agreed in advance
def kdf(x):
        return SHAKE128.new(x).read(32)

# In a real scenario, this key already exists
U_pub , U = MLDSA_65().keygen()

# In a real scenario, this key is received from the peer
# and it is verified as authentic
V , V_priv = MLDSA_65().keygen()

cipher_text_kem, shared_key  = MLDSA_65().encaps(V);

